In [2]:
## Import Libraries
# General system libraries
import os
import numpy as np
import pandas as pd
from time import time
from IPython.display import Image

# Multiprocessing
import multiprocessing

# DNA/RNA Analysis Libraries (Biopython, ViennaRNA, pysster) 
# Biopython Lib
import Bio
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio.Alphabet import generic_rna, generic_dna, generic_protein, IUPAC
# ViennaRNA Lib
import RNA
# pysster Lib
from pysster import utils
from pysster.Data import Data
from pysster.Grid_Search import Grid_Search
from pysster.One_Hot_Encoder import One_Hot_Encoder
from pysster.Alphabet_Encoder import Alphabet_Encoder
#RNAssp
import src.RNAssp.rna as rnassp

# Import TPOT libs
from tpot import TPOTRegressor

# Import Tensorflow
import tensorflow as tf

# Import Keras
from keras.models import Sequential, load_model
from keras.layers import Activation, Dropout, Flatten, Dense, Input
from keras.preprocessing import image
from keras.applications.imagenet_utils import decode_predictions, preprocess_input
from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras import optimizers
from keras import applications
from keras.models import Model
#from keras.utils import multi_gpu_model
from keras.callbacks import EarlyStopping
from keras_tqdm import TQDMNotebookCallback
from keras.utils.np_utils import to_categorical   

# Import sklearn libs
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import explained_variance_score, mean_absolute_error
from sklearn.metrics import mean_squared_error, mean_squared_log_error
from sklearn.metrics import median_absolute_error, r2_score

# Math & Visualization Libs
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats

# Warnings
import warnings

Using TensorFlow backend.


In [3]:
# Function definition to create complementary matrix of RNA molecule
def one_hot_special_complementarity_directional_matrix(m, show=False):
    """Produce complementarity matrix for the given RNA molecule.
    by Luis Soenksen 2019-02-01
    Complementary bases (according to Watson-Crick) are assigned the following values:
    G-C or C-G are assigned 3 = [0 0 0 1], 
    A-U or U-A are assigned 2 = [0 0 1 0],
    G-U or U-G are assigned 1 = [0 1 0 0], 
    NonWCpairs are assigned 0 = [1 0 0 0],
    G-C or C-G are assigned 3 = [0 0 0 1], 
    A-U or U-A are assigned 2 = [0 0 1 0],
    G-U or U-G are assigned 1 = [0 1 0 0], 
    NonWCpairs are assigned 0 = [1 0 0 0],

    Args:
        m: Molecule object.
        show (bool): Make a matrix plot of the result.

    Returns:
        p_oh: One-Hot Encoded Categorical Complementarity-directional Matrix
        p: Categorical(integer) Complementarity-directional Matrix
    """
    from keras.utils import to_categorical
    
    l = len(m.seq)
    p = np.zeros((l, l), dtype='float64')
    for i in range(l):
        for j in range(l):
            if m.seq[i] == 'G' and m.seq[j] == 'C' :
                p[i, j] = 6
            if m.seq[i] == 'A' and m.seq[j] == 'U' :
                p[i, j] = 4  
            if m.seq[i] == 'G' and m.seq[j] == 'U' :
                p[i, j] = 2
                
            # By default... if m.seq[i] == m.seq[j] ; p[i, j] = 0
            
            if m.seq[i] == 'C' and m.seq[j] == 'G':
                p[i, j] = 5
            if m.seq[i] == 'U' and m.seq[j] == 'A':
                p[i, j] = 3  
            if m.seq[i] == 'U' and m.seq[j] == 'G':
                p[i, j] = 1
                
    if show:
        %matplotlib ipympl
        fig = plt.figure()
        ax = fig.add_subplot(111)
        cmap = plt.get_cmap('jet', np.max(p)-np.min(p)+1)
        pos = ax.matshow(p, interpolation='nearest', cmap=cmap)
        ax.set_xticks(np.arange(l))
        ax.set_yticks(np.arange(l))
        ax.set_xticklabels([i for i in m.seq])
        ax.set_yticklabels([i for i in m.seq])
        
       
        # Add colorbar to make it easy to read the energy levels
        cbar = plt.colorbar(pos, ticks=np.arange(np.min(p),np.max(p)+1))
        plt.show()

    p_oh = to_categorical(p[:,:])
            
    return p_oh, p

In [8]:
# The RNA sequence
seq = "GAGUAGUGGAACCAGGCUAUGUUUGUGACUCGCAGACUAACA"
# compute minimum free energy (MFE) and corresponding structure
(ss, mfe) = RNA.fold(seq)
# print output
print ("%s\n%s [ %6.2f ]" % (seq, ss, mfe))

#Show
m = rnassp.Molecule(seq, ss)
seq_pair = rnassp.pair_matrix(m, show=True)
seq_p_oh, seq_p = one_hot_special_complementarity_directional_matrix(m, show=True)

GAGUAGUGGAACCAGGCUAUGUUUGUGACUCGCAGACUAACA
..(((((........)))))(((((((...)))))))..... [  -8.80 ]


FigureCanvasNbAgg()

FigureCanvasNbAgg()

In [35]:
#Check one-hot
seq_p_oh

array([[[1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        ...,
        [1., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 1.],
        [1., 0., 0., ..., 0., 0., 0.]],

       [[1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        ...,
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.]],

       [[1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        ...,
        [1., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 1.],
        [1., 0., 0., ..., 0., 0., 0.]],

       ...,

       [[1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        ...,
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0.